In [1]:
#https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia

In [2]:
import pandas as pd
import numpy as np

import glob
import matplotlib.pyplot as plt
import seaborn as sns


from tensorflow import keras
from tensorflow.keras import layers,callbacks,metrics
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from IPython.display import display
from IPython.display import Image as _imdis

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:19<00:00, 27.4MB/s]
100% 2.29G/2.29G [00:19<00:00, 126MB/s] 


In [5]:
from zipfile import ZipFile

dataset = "/content/chest-xray-pneumonia.zip"

with ZipFile(dataset,"r") as zip:
  zip.extractall()
  print("Data set has been extracted ")

Data set has been extracted 


In [6]:
train_path = "/content/chest_xray/train"
test_path = "/content/chest_xray/test"

In [7]:
from PIL import Image
import os
folder = train_path + "/NORMAL"

NORMAL = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder,f))]

print("working with {} images".format(NORMAL))
print("Images Example : ")

for i in range(0,10):
  print(NORMAL[i])
  display(_imdis(filename = folder + "/"+ NORMAL[i] ,width = 300,height = 300))

Output hidden; open in https://colab.research.google.com to view.

In [8]:
# create an architecture

In [9]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet',
                         include_top=False)

# add a global spatial average pooling layer
x = base_model.output

x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer

x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 200 classes

predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False


87910968/87910968 [==============================] - 0s 0us/step


In [10]:
# preprocessing the dataset

train_datagen = ImageDataGenerator(
                                rescale = 1/225.,
                                rotation_range = 40,
                                shear_range = 0.2,
                                zoom_range = 0.2,
                                horizontal_flip = True,
                                fill_mode = "nearest"
)

test_datagen = ImageDataGenerator(
                                rescale = 1/225.

)

In [11]:
# pass the data

train_data = train_datagen.flow_from_directory(
                                              train_path,
                                              target_size = (224,224),
                                              batch_size = 32,
                                              class_mode = "binary"
)

test_data = test_datagen.flow_from_directory(
                                              test_path,
                                              target_size = (224,224),
                                              batch_size = 32,
                                              class_mode = "binary"
)



Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [12]:
model.compile(
    optimizer ="adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, None, None, 32)       864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, None, None, 32)       96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, None, None, 32)       0         ['batch_normalization[0][0

In [16]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau


learning_rate_reduction = ReduceLROnPlateau(
                                    monitor = "val_accuracy",
                                    patience = 3,
                                    verbose = 1,
                                    factor = 0.6,
                                    min_lr = 0.0001
)

checkpoint = ModelCheckpoint(
    filepath = "InceptionV3_Model.h5",
    verbose = 1,
    save_best_only = True
)

callbacks = [checkpoint,learning_rate_reduction]

start = datetime.now()

history = model.fit(

                    train_data,
                    validation_data = test_data,
                    epochs = 5,
                    batch_size = 32,
                    callbacks = callbacks,
                    verbose = True
)

duration = datetime.now()-start

print("Training is completed")

Epoch 1/5
163/163 [==============================] - ETA: 0s - loss: 0.3171 - accuracy: 0.8854
Epoch 1: val_loss improved from inf to 0.42278, saving model to InceptionV3_Model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


163/163 [==============================] - 120s 733ms/step - loss: 0.3171 - accuracy: 0.8854 - val_loss: 0.4228 - val_accuracy: 0.8141 - lr: 0.0010
Epoch 2/5
163/163 [==============================] - ETA: 0s - loss: 0.1884 - accuracy: 0.9252
Epoch 2: val_loss did not improve from 0.42278
163/163 [==============================] - 111s 682ms/step - loss: 0.1884 - accuracy: 0.9252 - val_loss: 0.4765 - val_accuracy: 0.7949 - lr: 0.0010
Epoch 3/5
163/163 [==============================] - ETA: 0s - loss: 0.1719 - accuracy: 0.9294
Epoch 3: val_loss improved from 0.42278 to 0.39911, saving model to InceptionV3_Model.h5
163/163 [==============================] - 114s 703ms/step - loss: 0.1719 - accuracy: 0.9294 - val_loss: 0.3991 - val_accuracy: 0.8413 - lr: 0.0010
Epoch 4/5
163/163 [==============================] - ETA: 0s - loss: 0.1876 - accuracy: 0.9218
Epoch 4: val_loss improved from 0.39911 to 0.39390, saving model to InceptionV3_Model.h5
163/163 [==============================] - 113